## sentiment analysis - TFIDF feature weighting with RNN classification

`Term Weighting = TFIDF`

# Libraries

In [1]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #Count Vector Space Model
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics #Matrix Builder
from sklearn.metrics import accuracy_score  
from sklearn.model_selection import KFold #Import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout

from keras import Sequential
from keras.models import load_model

from sklearn.model_selection import KFold

import tensorflow as tf

# Load Dataset

_`preprocessed dataset`_

In [2]:
df = pd.read_csv('D:\kuliah\THE ONLY TA THINGS\DATA\cleaned_data_review.csv')
df

,content,stemming,label
0,halo min kenapa fitur teratas saya tidak ada m...,"['minimal', 'fitur', 'atas', 'minimal', 'tolon...",-1
1,Kenapa gak bisa live yaa,['live'],1
2,Ingin.tonton.vidio.kupu.kupu.mlm.,"['tonton', 'vidio', 'kupu', 'kupu']",1
3,Tiktok nya kayk kontoll,"['tiktok', 'kontoll']",1
4,Ini akun ketiga mudah²an g di banned lagi,"['akun', 'tiga', 'mudah', 'banned']",1
...,...,...,...
279,Tik tok ny langsung gak bisa di bukaa,['bukaa'],-1
280,Kkak,[],-1
281,Menyenangkan 😁,['senang'],1
282,Sangat bagus,['bagus'],1


In [3]:
df = df.dropna()
df = df.reset_index(drop=True)
# df = df[df.stemming != '[]']

In [4]:
df.isnull().sum()

content     0
stemming    0
label       0
dtype: int64

_`total label value`_

In [5]:
df['label'].value_counts()

 1    196
-1     74
 0     14
Name: label, dtype: int64

# TFIDF for feature weighting

#### split data test data train

In [6]:
X = df["stemming"]
y = df["label"] #data target

In [7]:
y.dtype

dtype('int64')

In [17]:
# # Convert text data to TF-IDF feature representation
# tfidf_vectorizer = TfidfVectorizer()
# X_train_vec = tfidf_vectorizer.fit_transform(X_train.apply(lambda x: ' '.join(x)))
# X_test_vec = tfidf_vectorizer.transform(X_test.apply(lambda x: ' '.join(x)))

tfidf_vect = TfidfVectorizer(use_idf = True ,max_features = 5000)
x = tfidf_vect.fit(df['stemming'])
TFIDF = x.transform(df['stemming'])
TFIDF = TFIDF.toarray()

# # Calculate the TF-IDF weights
# vectorizer = TfidfVectorizer()
# vectorizer.fit(X_train)
# x_train_tfidf = vectorizer.transform(X_train).toarray()
# x_test_tfidf = vectorizer.transform(X_test).toarray()

In [19]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(TFIDF, y, test_size=0.2, random_state=42)


In [20]:
print(TFIDF)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


_`Seperate label to its own array`_

In [21]:
label = []
for data in df['label']:
    label.append(data)
kolom = label.pop

In [14]:
def Average(lst):
    return sum(lst) / len(lst)

## implement w/ k-fold

In [ ]:
def create_model():
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

folds = range(10, 11)
for k in folds:
    kFoldCrossValidation = KFold(n_splits=k, random_state=0, shuffle = True)
    for train, test in kFoldCrossValidation.split(TFIDF, label):
        X_train, X_test = TFIDF[train], TFIDF[test]
        label = np.array(label)
        y_train, y_test = label[train], label[test]
        
        model = create_model()
        model.fit(np.expand_dims(X_train, axis=2), y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
        predictions = (model.predict(X_test) > 0.5).astype("int32")
        predictions = predictions.argmax(axis=-1)
    print(classification_report(y_test, predictions, digits=6))

In [ ]:
folds = range(2,11)
for k in folds:
    accuracy=[]
    kFoldCrossValidation = KFold(n_splits=k, random_state=0, shuffle = True)
    for train, test in kFoldCrossValidation.split(TFIDF, label):
        X_train, X_test = TFIDF[train], TFIDF[test]
        label = np.array(label)
        y_train, y_test = label[train], label[test]


        # embedding_size=32
        # model = Sequential()
        # model.add(Embedding(input_dim=5000, output_dim=embedding_size, input_length=X_train.shape[1]))
        # model.add(LSTM(100))
        # model.add(Dense(1, activation='sigmoid'))
        # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        model = Sequential()
        model.add(LSTM(units=128, input_shape=(X_train.shape[1], 1)))
        model.add(Dropout(0.5))
        model.add(Dense(units=1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        batch_size = 64
        num_epochs = 10
        model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(X_test, y_test))
        prediksi = model.predict(X_test)
        
        accuracy.append(accuracy_score(y_test, prediksi))
        
    print('Folds : %d | Accuracy : %.3f | Max, Min : %.3f, %.3f' 
          % (k, Average(accuracy), max(accuracy), min(accuracy)))
    print("\n")

In [22]:
# Define the k-fold cross-validation iterator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the RNN model
def build_model(input_dim):
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(input_dim, 1)))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [24]:
# Train and evaluate the RNN model using k-fold cross-validation
folds = range(10,11)

for k in folds:
    scores = []
    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train = np.array(y_train)
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        model = build_model(input_dim=X_train.shape[1])
        model.fit(np.expand_dims(X_train_fold, axis=2), y_train_fold, batch_size=64, epochs=5, verbose=0)
        loss, accuracy = model.evaluate(np.expand_dims(X_val_fold, axis=2), y_val_fold, verbose=0)
        scores.append(accuracy)
    print('Folds : %d | Accuracy : %.3f | Max, Min : %.3f, %.3f' 
            % (k, Average(scores), max(scores), min(scores)))
    print("\n")

print('Cross-validation accuracy:', np.mean(scores))

Folds : 10 | Accuracy : 0.057 | Max, Min : 0.111, 0.022


Cross-validation accuracy: 0.057294686511158946


In [25]:
# Evaluate the RNN model on the test set
loss, accuracy = model.evaluate(np.expand_dims(X_test, axis=2), y_test)
print('Test accuracy:', accuracy)

2/2 [==============================] - 1s 87ms/step - loss: 0.7107 - accuracy: 0.0175
Test accuracy: 0.017543859779834747


## Define RNN Model

In [ ]:
# embedding_size=32
# model = Sequential()
# model.add(Embedding(input_dim=5000, output_dim=embedding_size))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model = Sequential()
model.add(LSTM(units=128, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# batch_size = 64
# num_epochs = 5
# model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1)

model.fit(np.expand_dims(X_train, axis=2), y_train, batch_size=32, epochs=5)

In [ ]:
# Evaluate the model on the test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion matrix:')
print(conf_matrix)

In [ ]:
# Print the classification report
class_names = ['negative', 'neutral', 'positive']
report = classification_report(y_test, y_pred, target_names=class_names)
print('Classification report:')
print(report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the confusion matrix
plt.figure(figsize = (5, 5))
ax = sns.heatmap(conf_matrix, cmap = 'Blues',
                    linecolor = 'white',
                    linewidth = 1,
                    annot = True,
                    fmt = '',
                    xticklabels = ['negative', 'neutral', 'positive'],
                    yticklabels = ['negative', 'neutral', 'positive'])
ax.set_title("Confusion Matrix for TFIDF\n")
ax.set_xlabel("\nPredicted Values")
ax.set_ylabel("\nActual Values")
plt.show()